In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
# dataset example to see the columns
table=pd.read_csv('../Final/Berlin/2020-08-30_listings.csv')
table.columns

In [ ]:
pd.set_option("display.max_columns", 80) 
table.tail(6)

In [ ]:
pd.set_option("display.max_rows", 80) 
table.isnull().sum()

In [ ]:
dates=['2020-08-30','2020-06-13', '2020-05-14','2020-04-17','2020-03-17','2020-02-18','2020-01-10',
      '2019-12-11','2019-11-12','2019-10-16','2019-09-19','2019-08-12','2019-07-11','2019-06-08'
       ,'2019-05-14','2019-04-11','2019-03-08','2019-02-06','2019-01-14']#,'2018-12-10','2018-11-07']#,
       #'2018-10-10','2018-09-12','2018-08-14','2018-07-10','2018-06-09','2018-05-14','2018-04-12',
       #'2017-05-08','2017-04-08','2017-03-06','2017-02-10','2017-01-06','2016-12-08','2016-11-07']
       #'2016-10-04','2016-09-05','2016-08-04','2016-07-05','2016-06-03']
       #bigcrash
       #,'2016-05-03','2016-04-04','2016-02-04','2016-01-04', '2015-12-05','2015-11-08','2015-10-03']


In [ ]:
main_columns=['id','latitude','longitude','listing_url' ,'host_id','host_name','host_since', 'host_location',
              'room_type', 'neighbourhood_cleansed',
              'neighbourhood_group_cleansed','review_scores_location', 'price',
              'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
              'review_scores_checkin','review_scores_communication', 'review_scores_location','review_scores_value',
              'calculated_host_listings_count','number_of_reviews_ltm']

In [ ]:
def merging_dates(list_dates):
    df=pd.read_csv(f'../Final/Berlin/{list_dates[0]}_listings.csv', low_memory=False)[main_columns].copy()
    df=df.assign(date = pd.to_datetime(list_dates[0], format='%Y-%m-%d'))
    for date in list_dates[1:]:
        df2=pd.read_csv(f'../Final/Berlin/{date}_listings.csv', low_memory=False)[main_columns].copy()
        df2=df2.assign(date = pd.to_datetime(date, format='%Y-%m-%d'))
        df=pd.concat([df,df2])
        
    df=df.sort_values(by='date').reset_index(drop=True)
    
    return df
        

In [ ]:
# Actual merging 
# dont run if not necessary

newdf=merging_dates(dates)

In [ ]:
newdf

In [ ]:
###Cleaning some columns

In [ ]:
#round latitude and longitude

newdf[['latitude','longitude']]=newdf[['latitude','longitude']].round(decimals=5)

In [ ]:
#clean the price

newdf.price=pd.to_numeric(newdf.price.str.replace('$','').str.replace(',',''))

In [ ]:
airbnbs=newdf.groupby(['date', 'room_type'], as_index=False).count()

In [ ]:
airbnbs

In [ ]:
# have a look on the number of airbnbs data through time

fig, ax1 = plt.subplots(figsize=(15,5))



sns.lineplot(x = "date", y = "id", data = airbnbs, hue = "room_type",
             style = "room_type", markers = ["o", "<", "^",'>'],  legend="brief",)


ax1.set_ylabel('Number of airbnbs listings')
fig.suptitle("Airbnb in Berlin")
fig.savefig("Listings in Berlin.png")




In [ ]:
# save file 
# dont run if not necessary

name='from_jan2019'

newdf.to_csv(f'../Final/Berlin/merge_data_files/{name}.csv') 

In [ ]:
#### lock data from dataset depending on the variable

In [ ]:
#function for locking a dataset 
# column its a string
#argument is a list 


def lock_data(column, argument):
    df=pd.concat([newdf.loc[newdf[column]==i] for i in argument])
    return df

In [ ]:
#lock data from entire/apt
lockdf=lock_data('room_type', ['Entire home/apt']).to_csv('../Final/Berlin/merge_data_files/Entire_home_apt_2019_2020.csv') 

In [ ]:
lockdf

In [ ]:
# have a look on the number of airbnbs data through time

fig, ax1 = plt.subplots(figsize=(15,5))

airbnbs=lockdf.groupby('date').count()

ax1.plot(airbnbs['latitude'] , color='r')
ax1.set_ylabel('number of entire apartments on airbnbs',color='r')

In [ ]:
# dataframe for ploting


df_forplotting=newdf.copy()
df_forplotting['date']=df_forplotting['date'].apply(lambda x: x.strftime('%Y-%m'))


In [ ]:
df_forplotting.nunique()

In [ ]:
#ploting through time
#dont run if necessary

fig = px.density_mapbox(df_forplotting, lat='latitude', lon='longitude',
                        animation_frame="date", radius=1,
                        mapbox_style="stamen-terrain")
fig.update_layout(
    title={'text': "Airbnbs density map in Berlin "})
fig.show()
fig.write_html("../Final/density_map.html")